In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .appName("HELK JOIN") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [4]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [5]:
security_df = es_reader.load("logs-endpoint-winevent-security-*/")

In [6]:
security_df.createOrReplaceTempView("security_events")

In [7]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [8]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [22]:
process_hollowing = spark.sql(
'''
SELECT 
    c.host_name, 
    c.user_name,
    c.process_parent_name,
    a.process_name,
    d.process_target_name,
    a.logon_type,
    c.user_logon_id AS Sysmon_EID_1_LogonId,
    a.user_logon_id AS Security_EID_4624_TargetLogonId,
    d.process_granted_access
FROM security_events a
JOIN security_events b
    ON a.user_logon_id = b.user_logon_id
    AND b.event_id = 4688
JOIN sysmon_events c
    ON c.process_name = b.process_name
    AND c.event_id = 1
    AND c.user_name = b.user_name
    AND c.user_logon_id != b.user_logon_id
JOIN sysmon_events d
    ON c.process_guid = d.process_target_guid
    AND d.event_id = 10
    AND (d.process_granted_access & 2312) == 2312 -- 2312 = (0x908) process access rights for  PROCESS_VM_WRITE & PROCESS_VM_OPERATION & PROCESS_SUSPEND_RESUME & PROCESS_CREATE_PROCESS
    AND c.process_parent_guid = d.process_guid
    AND c.process_name = d.process_target_name
JOIN sysmon_events e
    ON e.process_guid = c.process_parent_guid
    AND e.event_id = 5
    AND e.process_name = d.process_name
    AND c.process_parent_name = e.process_name 
JOIN sysmon_events f
    ON f.process_guid = c.process_parent_guid
    AND f.event_id = 7
    AND f.process_path = c.process_parent_path
    AND f.process_path = f.module_loaded
    AND f.signed = "false"
WHERE 
    a.event_id = 4624
    AND a.logon_type = 11
    AND a.src_ip_addr is not null
'''
).show(10,False)

+------------------+---------+--------------------+------------+-------------------+----------+--------------------+-------------------------------+----------------------+
|host_name         |user_name|process_parent_name |process_name|process_target_name|logon_type|Sysmon_EID_1_LogonId|Security_EID_4624_TargetLogonId|process_granted_access|
+------------------+---------+--------------------+------------+-------------------+----------+--------------------+-------------------------------+----------------------+
|win10.marvel.local|thor     |gaudox.exe          |svchost.exe |explorer.exe       |11        |435609              |685481                         |2097151               |
|win10.marvel.local|thor     |gaudox.exe          |svchost.exe |explorer.exe       |11        |435609              |685481                         |2097151               |
|win10.marvel.local|thor     |gaudox.exe          |svchost.exe |explorer.exe       |11        |435609              |435917                  